<a href="https://colab.research.google.com/github/dacq-trap/MachineLearningWorkshop/blob/main/%E7%AC%AC3%E5%9B%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 機械学習講習会第三回

## 演習の流れ
1. データセットをロード
1. pandasを使ってデータセットを加工
1. numpyを使って線形回帰を解く

前回演習で使用したMNISTでは、データセット加工の部分が既に行われており機械学習モデルに直接投げることが出来ました。

この演習では、収集した生データを機械学習に適した形に加工することを目標に進めていきます。


データセットにはCalifornia Housing Prices dataset：
https://www.kaggle.com/datasets/camnugent/california-housing-prices
を一部加工したものを使います。

今回はこのデータからmedian_house_value(住宅価格の中央値)を線形回帰で求めてみたいと思います。

全体的に「scikit-learn、Keras、TensorFlowによる実践機械学習」をパクっています。


## 1. データセットのロード


ダウンロード

In [ ]:
import os
import tarfile
import urllib.request
import pandas as pd

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def load_housing_data():
    if not os.path.isdir(HOUSING_PATH):
        os.makedirs(HOUSING_PATH)
    tgz_path = os.path.join(HOUSING_PATH, "housing.tgz")
    urllib.request.urlretrieve(HOUSING_URL, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=HOUSING_PATH)
    housing_tgz.close()

    csv_path = os.path.join(HOUSING_PATH, "housing.csv")
    return pd.read_csv(csv_path)


housing = load_housing_data()

中身確認

pandasで読み込んだデータはDataFrameという型となります。

データセットを保持・加工するのに適しています。

In [ ]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [ ]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


データ数が20640で、total_bedroomsに欠損があり、ocean_proximityが値ではないことが分かります。

|ラベル|日本語訳|
|-|-|
|longitude|経度|
|latitude|緯度|
|housing_median_age|住宅築年数(中央値)|
|total_rooms|総部屋数|
|total_bedrooms|総ベッド数|
|population|人口|
|households|世帯数|
|median_income|収入(中央値)|
|median_house_value|住宅価格(中央値)|
|ocean_proximity|海との位置関係|


In [ ]:
housing["ocean_proximity"].value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

ocean_proximityはカテゴリ型のようです。



In [ ]:
housing.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


他の数値型も特徴量ごとに数字のスケールがかなり違うことが分かります。


## 2. pandasを使ってデータセットを加工

pandasでデータ加工と言っていますが、自分はpandasを全然使わない派の人間なのでscikit-learnが所々出てきます。

ごめんね

### 分割

データセットを加工する前に訓練用データセットとテスト用データセットに分けます

sklearnのtrain_test_split関数を使って、訓練セット：テストセット＝8:2に分割します

本当は訓練セット、検証セット、テストセットに分ける必要がありますが、ここでは検証セットは省略します

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)


In [ ]:
print(len(train_set), len(test_set))

16512 4128


In [ ]:
x_train_raw = train_set.drop("median_house_value", axis=1)
y_train_raw = train_set["median_house_value"]
x_test_raw = test_set.drop("median_house_value", axis=1)
y_test_raw = test_set["median_house_value"]

無事分割出来ました

train_test_splitでrandom_state=42として乱数を固定していますが、これは実行する度にtrain_setとtest_setが混ざるのを防ぐため。

### 欠損値処理

total_bedroomsに欠損値があるので処理します

欠損値の処理方法は3通りあり
1. 欠損値があるインスタンスを削除する
2. 欠損値がある特徴量を削除
3. 何らかの値で穴埋めする

穴埋めする場合、数値型なら平均値や中央値など、カテゴリ型なら最頻値などで補完します。

ここでは、中央値で補完することにします。

In [ ]:
total_bedrooms_median = x_train_raw["total_bedrooms"].median()
x_train_raw["total_bedrooms"].fillna(total_bedrooms_median, inplace=True)

x_train_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 14196 to 15795
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16512 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
 8   ocean_proximity     16512 non-null  object 
dtypes: float64(8), object(1)
memory usage: 1.3+ MB


ここで補完に使った中央値は保存しておく必要があります。

テストセットや検証セットの欠損値に対して、ここで保存した中央値で補完します。

もし余裕があれば、以下を参考にインスタンスを削除するパターンも試してみましょう

https://note.nkmk.me/python-pandas-nan-fillna/

#### 答え






```
x_train_raw.dropna(subset=["total_bedrooms"])    # 1. 欠損があるインスタンス削除
x_train_raw.drop("total_bedrooms", axis=1)       # 2. total_bedrooms自体を削除
```

### カテゴリ型の処理

In [ ]:
x_train_raw["ocean_proximity"].value_counts()

<1H OCEAN     7341
INLAND        5227
NEAR OCEAN    2086
NEAR BAY      1854
ISLAND           4
Name: ocean_proximity, dtype: int64

ocean_proximityがカテゴリで表されているが、このままだと扱いづらいので数値に置き換えます。

1H OCEANを0、INLANDを1、NEAR OCEANを2、...  と置き換えると、1H OCEANとINLANDが似ていて、ISLANDとは大きく異なるというふうに機械学習モデルが勘違いしてしまいます。

そこで、1H OCEANを[1,0,0,0,0]、INLANDを[0,1,0,0,0]、NEAR OCEANを[0,0,1,0,0]、...というふうに行列に置き換えることにします。

これをワンホットエンコーディングと言います。


面倒なのでcategory_encodersのOneHotEncoderを使います。

fitでデータに適合し、transformでデータを変換します。

fit_trainsformはこれを合わせたものです。

訓練セットに対してはfit_transform、テストセットに対してはtransformを使います。

In [ ]:
!!pip install category_encoders

['Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/',
 'Collecting category_encoders',
 '  Downloading category_encoders-2.5.0-py2.py3-none-any.whl (69 kB)',
 '\x1b[?25l',
 '\x1b[K     |████▊                           | 10 kB 11.4 MB/s eta 0:00:01',
 '\x1b[K     |█████████▌                      | 20 kB 13.3 MB/s eta 0:00:01',
 '\x1b[K     |██████████████▎                 | 30 kB 6.3 MB/s eta 0:00:01',
 '\x1b[K     |███████████████████             | 40 kB 7.7 MB/s eta 0:00:01',
 '\x1b[K     |███████████████████████▊        | 51 kB 9.1 MB/s eta 0:00:01',
 '\x1b[K     |████████████████████████████▌   | 61 kB 10.5 MB/s eta 0:00:01',
 '\x1b[K     |████████████████████████████████| 69 kB 2.1 MB/s ',
 '\x1b[?25hRequirement already satisfied: scikit-learn>=0.20.0 in /usr/local/lib/python3.7/dist-packages (from category_encoders) (1.0.2)',
 'Requirement already satisfied: pandas>=1.0.5 in /usr/local/lib/python3.7/dist-packages (from category_enco

In [ ]:
import category_encoders as ce

cat_encoder = ce.OneHotEncoder(cols=["ocean_proximity"])

x_train_raw = cat_encoder.fit_transform(x_train_raw)
x_train_raw

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_1,ocean_proximity_2,ocean_proximity_3,ocean_proximity_4,ocean_proximity_5
14196,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596,1,0,0,0,0
8267,-118.16,33.77,49.0,3382.0,787.0,1314.0,756.0,3.8125,1,0,0,0,0
17445,-120.48,34.66,4.0,1897.0,331.0,915.0,336.0,4.1563,1,0,0,0,0
14265,-117.11,32.69,36.0,1421.0,367.0,1418.0,355.0,1.9425,1,0,0,0,0
2271,-119.80,36.78,43.0,2382.0,431.0,874.0,380.0,3.5542,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,-117.96,33.78,35.0,1330.0,201.0,658.0,217.0,6.3700,0,0,1,0,0
11964,-117.43,34.02,33.0,3084.0,570.0,1753.0,449.0,3.0500,0,1,0,0,0
5390,-118.38,34.03,36.0,2101.0,569.0,1756.0,527.0,2.9344,0,0,1,0,0
860,-121.96,37.58,15.0,3575.0,597.0,1777.0,559.0,5.7192,0,0,1,0,0


### スケーリング

特徴量のスケールを確認します。

In [ ]:
x_train_raw.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_1,ocean_proximity_2,ocean_proximity_3,ocean_proximity_4,ocean_proximity_5
count,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000
mean,-119.582290,35.643149,28.608285,2642.004784,538.496851,1426.453004,499.986919,3.880754,0.126332,0.316558,0.444586,0.112282,0.000242
std,2.005654,2.136665,12.602499,2174.646744,419.007096,1137.056380,380.967964,1.904294,0.332234,0.465147,0.496935,0.315723,0.015563
min,-124.350000,32.550000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-121.810000,33.930000,18.000000,1454.000000,296.750000,789.000000,280.000000,2.566700,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-118.510000,34.260000,29.000000,2129.000000,437.000000,1167.000000,410.000000,3.545800,0.000000,0.000000,0.000000,0.000000,0.000000
75%,-118.010000,37.720000,37.000000,3160.000000,647.000000,1726.000000,606.000000,4.773175,0.000000,1.000000,1.000000,0.000000,0.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,1.000000,1.000000,1.000000,1.000000,1.000000


項目によって値のスケールが違います。

スケールが違うと、スケールが大きい特徴量にモデルが影響されやすくなる事があるので、スケールを合わせる必要があります。

よく使われる手法は最小最大スケーリングと標準化です。

今回は値を0~1の範囲に丸める最小最大スケーリングをしてみます。


In [ ]:
max = train_set["longitude"].max()
min = train_set["longitude"].min()
train_set["longitude"] = (train_set["longitude"] - min) / (max - min)

train_set["longitude"].describe()

count    16512.000000
mean         0.474871
std          0.199766
min          0.000000
25%          0.252988
50%          0.581673
75%          0.631474
max          1.000000
Name: longitude, dtype: float64

流石にこれを全部の特徴量に行うのは面倒なので、scikit-learnのMinMaxScaler関数を使って楽します。

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train_raw = scaler.fit_transform(x_train_raw)

In [ ]:
x_train_raw[0:10]

array([[0.72908367, 0.01702128, 0.62745098, 0.0794547 , 0.09714463,
        0.06437961, 0.10228581, 0.19032151, 1.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.61653386, 0.12978723, 0.94117647, 0.08596572, 0.12197393,
        0.0367443 , 0.12415721, 0.22845202, 1.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.38545817, 0.22446809, 0.05882353, 0.04819675, 0.05121043,
        0.02556125, 0.05508962, 0.25216204, 1.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.72111554, 0.01489362, 0.68627451, 0.03609034, 0.05679702,
        0.03965918, 0.05821411, 0.09948828, 1.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.45318725, 0.45      , 0.82352941, 0.06053207, 0.06672874,
        0.02441212, 0.06232528, 0.21063847, 0.        , 1.        ,
        0.        , 0.        , 0.        ],
       [0.24800797, 0.51808511, 0.37254902, 0.12793123, 0.12523277,
        0.07545055, 0.13155

### テストセットにも同じ変換を訓練セットと同じ変換をする

**演習**　訓練セットへの変換を参考に、テストセットにも欠損値処理、OneHotエンコード、スケーリングをしてみましょう。

ヒント：欠損値には訓練セットで補完したときの値を使います。
OneHotエンコードとスケーリングではcat_encoderとscalerが変換に必要な値を持っているので、transformで変換できます。

In [ ]:
# 欠損値処理
#x_test_raw = ???

# OneHotエンコード
#x_test_raw = ???

# スケーリング
#x_test_raw = ???

#### 答え

In [ ]:
# 欠損値処理
x_test_raw["total_bedrooms"].fillna(total_bedrooms_median, inplace=True)

# OneHotエンコード
x_test_raw = cat_encoder.transform(x_test_raw)

# スケーリング
x_test_raw = scaler.transform(x_test_raw)


## モデル訓練

In [ ]:
x_train = x_train_raw
y_train = y_train_raw.to_numpy(dtype="float64")
x_test = x_test_raw
y_test = y_test_raw.to_numpy(dtype="float64")

### sklearnのLinearRegression関数に投げてみる

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(x_train, y_train)                         # 線形モデルの重みを学習


LinearRegression()

係数と切片表示

In [ ]:
print('coefficient = ', lr.coef_)
print('intercept = ', lr.intercept_)

coefficient =  [ -269456.26465959  -239402.50926886    56211.43928254  -236753.55975715
   662374.86390017 -1361971.12838034   293424.9896976    572380.53483714
   -15495.4427888    -58713.23902328   -18926.58286195   -24063.22507943
   117198.48975346]
intercept =  272583.34148021037


In [ ]:
from sklearn.metrics import mean_absolute_error

print("MAE train data:", mean_absolute_error(y_train, lr.predict(x_train)))
print("MAE test data:", mean_absolute_error(y_test, lr.predict(x_test)))


for i in range(10):
    print(y_test[i], lr.predict(x_test[i].reshape(-1,13)))

MAE train data: 49594.84209472436
MAE test data: 50670.48923565361
47700.0 [54261.02768976]
45800.0 [124430.91772796]
500001.0 [255694.95828244]
218600.0 [268208.01035997]
278000.0 [262975.01360647]
158700.0 [139811.88274638]
198200.0 [290871.00270495]
157500.0 [228470.45516607]
340000.0 [256712.36440083]
446600.0 [408129.43722564]


平均絶対誤差がものすごくでかいですが、これは目的変数であるmedian_house_valueだけスケーリングしていないないからです

median_house_valueは最大値：15,000、最大値：500,000なので決して良くないですが

### numpyで線形回帰を実装

多変数の線形回帰を実装するのは難易度が高すぎるので、1変数で最小二乗法で解ける線形回帰を実装します。

In [ ]:
x_train_single = x_train[:,7]
x_test_single = x_test[:,7]

**課題**　式をy=ax+bとして、xとyを渡されたときにaとbを返す関数linear(x,y)を定義してみましょう

最小二乗法のヒント：第2回の資料(**訂正が入っているので再ダウンロード推奨**)

numpyのヒント：https://naoyat.hatenablog.jp/entry/2011/12/29/021414

チートシート(x,y,z:numpy配列、c:実数、i:添字、n:xの要素数)

$x_i + y_i$ ： ```x + y```　　# 配列の各要素を足す

$x * c$ ： ```x*c```　　# 配列を実数倍する

$∑^n_{i=1}x_i$ ： ```np.sum(x)```　　# 配列をすべて足す

$z_i = x_i^2$ ： ```z = np.square(x)```　　# 配列の各要素を2乗する

$z_i = x_i * y_i$ ： ```z = np.dot(x,y)```　　# 配列の各要素を掛け合わせる



In [ ]:
import numpy as np

def linear(x, y):
    n = x.size
    a = 0
    b = 0
    # --- ここに実装してください ---

    return a,b

def predict(x, a, b):
    return x * a + b

a, b = linear(x_train_single, y_train)


In [ ]:
print("coefficient = ", a)
print("intercept = ", b)

for i in range(10):
    print(y_test[i], predict(x_test_single[i], a, b))

coefficient =  0
intercept =  0
47700.0 0.0
45800.0 0.0
500001.0 0.0
218600.0 0.0
278000.0 0.0
158700.0 0.0
198200.0 0.0
157500.0 0.0
340000.0 0.0
446600.0 0.0


#### 答え



In [ ]:
import numpy as np

def linear(x, y):
    n = x.size
    a = 0
    b = 0
    # aの分子
    a = n * np.sum(np.dot(x,y)) - np.sum(x) * np.sum(y)
    # aの分母
    a /= n * np.sum(np.square(x)) - np.sum(x)**2

    # bの分子
    b = np.sum(np.square(x)) * np.sum(y) - np.sum(x) * np.sum(np.dot(x,y))
    # bの分母
    b /= n * np.sum(np.square(x)) - np.sum(x)**2
    return a, b

def predict(x, a, b):
    return x*a + b

a, b = linear(x_train_single, y_train)


In [ ]:
print('coefficient = ', a)
print('intercept = ', b)

print("MAE train data:", mean_absolute_error(y_train, predict(x_train_single, a, b)))
print("MAE test data:", mean_absolute_error(y_test, predict(x_test_single, a, b)))

for i in range(10):
    print(y_test[i], predict(x_test_single[i], a, b))


coefficient =  608049.202980165
intercept =  65422.46048104547
MAE train data: 62495.076556687294
MAE test data: 62990.86530093762
47700.0 114958.91676995659
45800.0 150606.8821396369
500001.0 190393.71844448656
218600.0 285059.383451019
278000.0 200663.3181610313
158700.0 242165.24890608786
198200.0 257647.22610228357
157500.0 199229.18051176288
340000.0 245893.16811719784
446600.0 384677.4360709609


#### sklearnで確認

In [ ]:

lr2 = LinearRegression()

lr2.fit(x_train_single.reshape(-1,1), y_train)          

LinearRegression()

In [ ]:
print('coefficient = ', lr2.coef_)
print('intercept = ', lr2.intercept_)

print("MAE train data:", mean_absolute_error(y_train, lr2.predict(x_train_single.reshape(-1,1))))
print("MAE test data:", mean_absolute_error(y_test, lr2.predict(x_test_single.reshape(-1,1))))

for i in range(10):
    print(y_test[i], lr2.predict(x_test_single[i].reshape(-1,1)))



coefficient =  [608049.20298016]
intercept =  65422.46048104574
MAE train data: 62495.07655668728
MAE test data: 62990.86530093761
47700.0 [114958.91676996]
45800.0 [150606.88213964]
500001.0 [190393.71844449]
218600.0 [285059.38345102]
278000.0 [200663.31816103]
158700.0 [242165.24890609]
198200.0 [257647.22610228]
157500.0 [199229.18051176]
340000.0 [245893.1681172]
446600.0 [384677.43607096]


#### RandomForestに突っ込んでみた

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()

rf.fit(x_train, y_train)    
 

RandomForestRegressor()

In [ ]:

print("MAE train data:", mean_absolute_error(y_train, rf.predict(x_train)))
print("MAE test data:", mean_absolute_error(y_test, rf.predict(x_test)))


for i in range(10):
    print(y_test[i], rf.predict(x_test[i].reshape(-1,13)))

MAE train data: 11647.493710634688
MAE test data: 31489.578425387597
47700.0 [49214.]
45800.0 [69059.]
500001.0 [462127.38]
218600.0 [261741.01]
278000.0 [263803.]
158700.0 [162628.]
198200.0 [217130.]
157500.0 [167383.]
340000.0 [276377.02]
446600.0 [480915.69]
